In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv('drive/My Drive/signate/takeda/dataset/train.csv')
dftest = pd.read_csv("drive/My Drive/signate/takeda/dataset/test.csv")

print(df.shape)
print(dftest.shape)

In [0]:
# 欠損値の処理
df_missing_value = pd.DataFrame()
dftest_missing_value = pd.DataFrame()

for flag, name in zip(df.isnull().any(), df.columns):
  if flag == True:
    df_missing_value = pd.concat(objs=[df_missing_value, df[name]], axis=1)
    dftest_missing_value = pd.concat(objs=[dftest_missing_value, dftest[name]], axis=1)
    df = df.drop(name, axis=1)
    dftest = dftest.drop(name, axis=1)
    
for flag, name in zip(dftest.isnull().any(), dftest.columns):
  if flag == True:
    df_missing_value = pd.concat(objs=[df_missing_value, df[name]], axis=1)
    dftest_missing_value = pd.concat(objs=[dftest_missing_value, dftest[name]], axis=1)
    dftest = dftest.drop(name, axis=1)
    df = df.drop(name, axis=1)

same_colums = []
same_columns = df_missing_value.columns & dftest_missing_value.columns
print(len(same_colums))
print(df_missing_value.shape)
print(dftest_missing_value.shape)

0
(0, 0)
(0, 0)


In [0]:
df_Score = df['Score']
df = df.drop('Score', axis=1)
dftest_id = dftest['ID']

print(df.shape)
print(dftest.shape)

(13731, 3806)
(13732, 3806)


In [0]:
# ステップワイズ法
import statsmodels.api as sm

model = sm.OLS(df_Score, sm.add_constant(df))
result = model.fit()

for name, num in zip(df.columns, result.pvalues):
  if num > 0.05:
    df = df.drop(name, axis=1)
    dftest = dftest.drop(name, axis=1)
    
print(df.shape)
print(dftest.shape)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

X = df.as_matrix()
y = df_Score.as_matrix()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# 標準化(重回帰時のみ使用)
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

#clf = lgb.LGBMRegressor()         # 0.540 # 0.537
#clf = GradientBoostingRegressor() # 0.472
#clf = xgb.XGBRegressor()          # 0.472
#clf = RandomForestRegressor()     # 0.441 

clf = linear_model.LinearRegression()  # -998056659182317207552.000, -4341099214631143150190592.000
#clf = Lasso()                         # -0.001
#clf = Ridge()                         # 0.182

#clf.fit(X_train, y_train)

clf.fit(X_train_std, y_train)

print('決定係数(train):{:.3f}'.format(clf.score(X_train_std, y_train)))
print('決定係数(test):{:.3f}'.format(clf.score(X_test_std, y_test)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


決定係数(train):0.369
決定係数(test):-4341099214631143150190592.000


In [0]:
clf.fit(X, y)

XX = dftest.as_matrix()

dftest_Score = pd.DataFrame(clf.predict(XX))

dftest_Score = dftest_Score.round()
dftest_Score = pd.concat(objs=[dftest_id, dftest_Score], axis=1)
dftest_Score.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,ID,0
0,1,2.0
1,2,3.0
2,3,3.0
3,4,2.0
4,6,3.0


In [0]:
dftest_Score.to_csv('submit_Linear.csv', index=False, header=False)